In [1]:
## Import der Module
# Pyhton Version 3.11.1
# Selenium Version 4.9.1
# webdriver_manager Version 3.8.6
# time
# random
# pandas Version  2.0.1
# datetime Version 5.1
# sqlalchemy Version 2.0.13
# os 
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import random
import pandas as pd
import datetime
from sqlalchemy import create_engine
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
#----------------------------------------------------------------------------------
## Hilfsfunktionen
# Log File
def schreibe_log_file(log_eintrag):
    datei='log-LinkedIn.txt' # Name der Log-Datei
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Überprüfen, ob die Log-Datei bereits existiert
    if not os.path.exists(datei):
        # Wenn die Log-Datei nicht existiert, wird sie erstellt und der Log-Eintrag wird hineingeschrieben
        with open(datei, 'w') as file:
            file.write(f"{timestamp}, {log_eintrag}" + '\n')
    else:
        # Wenn die Log-Datei bereits existiert, wird der Log-Eintrag an das Ende der Datei angehängt
        with open(datei, 'a') as file:
            file.write(f"{timestamp}, {log_eintrag}" + '\n')
            
# E-Mail - Benachrichtigung bei Fehlern
def schreibe_e_mail(message, Subject="fehlerhaft"):
    # SMTP-Server-Konfiguration
    host = "smtp.web.de"
    port = 587
    log_in_id = "dc-jobs"
    passwort = "DatacraftKurs0822!"

    # Erstellen der E-Mail-Nachricht
    msg = MIMEMultipart()
    msg['From'] = log_in_id + "@web.de" # Absender
    msg['To'] = "jobs@data-craft.de" # Empfänger
    msg['Subject'] = f'LinkedIn Abfrage {Subject}' # Betreff der E-Mail

    msg.attach(MIMEText(message, 'plain')) # Hinzufügen des Nachrichtentextes zur E-Mail

    # Verbindung zum SMTP-Server herstellen und E-Mail senden
    with smtplib.SMTP(host=host, port=port) as mail:
        mail.starttls() # Starte TLS-Verschlüsselung
        mail.login(log_in_id, passwort) # Anmeldung am SMTP-Server
        mail.send_message(msg) # E-Mail senden

# random Wartezeit zum für seitenaufbau oder so
def wartezeit(zeit=3):
    """
    Diese Funktion fügt eine zufällige Wartezeit hinzu, bevor sie fortgesetzt wird.

    :param zeit: Die maximale Wartezeit in Sekunden (Standardwert ist 3).
    :return: None
    """
    time.sleep(random.randint(1, zeit))

# Da LinkedIn (zwecks Erkennung auf Bot´s) gerne eine Anmeldung möchte diese Abfangen
def anmeldemaske_finden(driver):
    # Anmeldemaske finden und text Abspeichern
    try:
        return driver.find_element(By.CLASS_NAME, 'authwall-join-form__title').text == 'Mitglied werden'
    except:
        return False
    
#---------------------------------------------------------------------------
## SQL-Server verbindung erstellen
# SQL-Server-Konfiguration
#host="datacraft-db.cf3hyz1fwdiw.eu-central-1.rds.amazonaws.com"
host='datacraft-jobs.postgres.database.azure.com'
database="postgres"
user="jobs_rw"
password="01QT3X48xCi4KXUckxDq"
tabelle_Rohdaten="jobs.rohdaten"

# Erstelle eine SQL-Verbindung mit der Datenbank
connection = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}/{database}')
#----------------------------------------------------------------------------
## Der Scraper
def scraper(jobtitel, suchort='Deutschland'):
    '''
    Der eigentliche Scraper für die Seite
    
    :param jobtitel: Der Titel des Jobs, nach dem gesucht werden soll.
    :param suchort: Der Ort, an dem nach Jobs gesucht werden soll. Standardwert ist 'Deutschland'.
    '''
    
    schreibe_log_file(f'suche nach {jobtitel}')
    
    # ChromeOptions erstellen
    chrome_options = webdriver.ChromeOptions()
    #Headless-Modus aktivieren
    #chrome_options.add_argument("--headless")

    # Browserfenster öffnen nach Einstellung in den Optionen
    try:
        driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)
        schreibe_log_file('Browser geoeffnet')
    except:
        schreibe_log_file('Browser konnte nicht geoeffnet werden')
        schreibe_e_mail(f'Browser konnte nicht geoeffnet werden: -- {datetime.datetime.now().strftime("%d.%m.%Y, %H:%M:%S")} --')

    # Webseite aufrufen
    wartezeit(1)
    driver.get("https://de.linkedin.com/jobs/search?keywords=&location=&geoId=&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0")

    # Fenster Maximieren
    driver.maximize_window() # evtl. im Headless-Modus nicht machbar
    #--------------------------------------------------------------------------
    ## Cookies akzeptieren
    wartezeit(1)
    driver.find_element(By.XPATH, '//*[@id="artdeco-global-alert-container"]/div/section/div/div[2]/button[1]').click()
    wartezeit(1)
    
    if anmeldemaske_finden(driver=driver) == True:
        schreibe_log_file(f'als Bot erkannt')
        # Schließen des Browsers
        wartezeit(1)
        driver.quit()
        schreibe_log_file('Browser geschlossen')
    else:
        ## Suchfelder finden, anwählen und befüllen
        # Suchfeld Jobbezeichnung -> finden und befüllen
        suchfeld_jobtitel=driver.find_element(By.XPATH,'//*[@id="job-search-bar-keywords"]')
        suchfeld_jobtitel.send_keys(jobtitel)#"jobtitel" wird an die Funktion übergeben
        wartezeit(2)

        # Suchfeld Ort -> finden und befüllen
        suchfeld_ort=driver.find_element(By.XPATH,'//*[@id="job-search-bar-location"]')
        suchfeld_ort.clear()
        wartezeit(2)
        suchfeld_ort.send_keys(suchort)#"suchort" wird an die Funktion übergeben
        time.sleep(2)
        
        # Anfrage mit Enter/Return abschicken
        suchfeld_ort.send_keys(Keys.RETURN)
        time.sleep(random.randint(5,10))

        # Überprüfen ob der Suchort noch mit der Eingabe übereinstimmt
        
        try:
            jobtitel_feld=driver.find_element(By.XPATH,'//*[@id="job-search-bar-keywords"]').get_attribute('value')
            suchort_feld=driver.find_element(By.XPATH,'//*[@id="job-search-bar-location"]').get_attribute('value')
        except:
            schreibe_log_file(f'als Bot erkannt: {link}')
            return
                
        if suchort_feld != suchort or jobtitel_feld != jobtitel:
            schreibe_log_file(f'Der Suchort oder Jobtitel wurde von LinkedIn abgeaendert, evtl. wurden wir als BOT erkannt!!!')
            schreibe_e_mail(f'Der Suchort oder Jobtitel wurde von LinkedIn abgeaendert, evtl. wurden wir als BOT erkannt!!!\n\n>>>{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}<<<\n\nJobtitel: {jobtitel}\nSuchort: {suchort}\n\nAutomatisch abgeändert in:\nJobtitel: {jobtitel_feld}\nSuchort: {suchort_feld}')
            wartezeit(1)
            driver.quit()
            schreibe_log_file('Browser geschlossen')
            #break
        
        else:

            ## Log-In Fenster entfernen
            driver.find_element(By.XPATH, '/html/body/div[3]/button').click()
            time.sleep(random.randint(5,10))
            #-------------------------------------------------------------------------
            ## Seite für weitere Stellenanzeigen nach unten Scrollen
            for i in range(6):
                driver.execute_script("window.scrollBy(0, document.body.scrollHeight);")
                time.sleep(random.randint(3.0,4.0))
            #-------------------------------------------------------------------------
            link_liste_scraped=[]

            anzeigen=driver.find_elements(By.CLASS_NAME, 'base-card__full-link')

            for anzeige in anzeigen:
                link_liste_scraped.append(anzeige.get_attribute('href'))
            #-------------------------------------------------------------------------
            # überprüfen ob link_liste_scraped leer ist da evtl. die Class name geändert wurde
            if len(link_liste_scraped) == 0:
                schreibe_log_file('Keine Links auf gefunden')
                schreibe_e_mail('Keine Links auf Webseite gefunden:\n\n Hinweis: Classenname ueberpruefen fuer alle Stellenanzeigen!')
                return
            else:
                pass
            #------------------------------------------------------------------------
            # Duplikate entfernen
            link_liste_scraped_ohne_duplikate = list(set(link_liste_scraped))
            schreibe_log_file(f'Es wurden {len(link_liste_scraped)-len(link_liste_scraped_ohne_duplikate)} Duplikate in der "Link Liste" entfernt')
            schreibe_log_file("link liste erstellt")
            #------------------------------------------------------------------------
            # Lese die bestehenden URLs aus der Datenbank
            existing_urls = pd.read_sql_query(f"SELECT url FROM {tabelle_Rohdaten}", connection)
            schreibe_log_file(f'{len(existing_urls)} in der datenbak bereits vorhanden')
            
            # Filtere den DataFrame, um nur neue URLs zu behalten
            link_liste = [url for url in link_liste_scraped_ohne_duplikate if url not in existing_urls['url'].values]
            #------------------------------------------------------------------------
            seiten_inhalt_html_liste = []
            seiten_inhalt_liste = []
            URL_Liste = []
            Datum_Liste = []

            try:
                schreibe_log_file('Abfrage begonnen')
                for link in link_liste:
                    
                    # Seite Aufrufen
                    driver.get(link)
                    wartezeit()
                    
                    if anmeldemaske_finden(driver=driver) == False:
                    
                        #Angaben erweitern
                        try:
                            driver.find_element(By.XPATH, '//*[@id="main-content"]/section[1]/div/div/section[1]/div/div/section/button[1]').click()
                        except:
                            pass
                        wartezeit(2)
                        
                        # Listen befüllen
                        seiten_inhalt_html_liste.append(driver.find_element(By.CLASS_NAME, 'details').get_attribute('innerHTML'))
                        seiten_inhalt_liste.append(driver.find_element(By.CLASS_NAME, 'details').text)
                        URL_Liste.append(link)
                        Datum_Liste.append(datetime.datetime.now())
                        schreibe_log_file(f'Daten wurden gezogen: {link}')
                        time.sleep(random.randint(2,10))
                    
                    else:
                        schreibe_log_file(f'als Bot erkannt: {link}')
            except:
                schreibe_log_file("Abfrage abgebrochen !!!")
                schreibe_e_mail(f"Die Abfrage wurde abgebrochen. -- {datetime.datetime.now()} --")
            schreibe_log_file('Abfrage beendet')
            #-------------------------------------------------------------------------
            # Schließen des Browsers
            wartezeit(1)
            driver.quit()
            schreibe_log_file('Browser geschlossen')
            #-------------------------------------------------------------------------
            #Überprüfen ob die Listen gleich lang sind und DataFrame erstellen
            if len(set(map(len, [seiten_inhalt_html_liste,
                                seiten_inhalt_liste,
                                URL_Liste,
                                Datum_Liste]))) > 1:
                schreibe_log_file('Listen sind verschieden lang')
                schreibe_e_mail('Listen sind nicht gleich lang')
                return
            else:
                # DataFrame erstellen
                df = pd.DataFrame({"seite": "linkedin",
                                "seiten_inhalt_html": seiten_inhalt_html_liste,
                                "seiten_inhalt": seiten_inhalt_liste, 
                                "url": URL_Liste, 
                                "datum":Datum_Liste,
                                "storno": False})
            #------------------------------------------------------------------------
            ## Daten in die Datenbank einfügen
            # Schreibe den bereinigten DataFrame in die Datenbank
            df.to_sql(name=tabelle_Rohdaten.split('.')[1],
                    schema=tabelle_Rohdaten.split('.')[0],
                    con=connection, if_exists='append', index=False, )
            schreibe_log_file(f'Es wurden {len(df)} Daten von LinkedIn hinzugefuegt')
        
        # Verbindung zur SQL-Datenbank schließen
        connection.dispose()
        #else:
        #    schreibe_log_file(f'als Bot erkannt: {link}')
print("fertig")

fertig


In [2]:
# Ich arbeite mit zwei Computern, und leider hat die Datei für die Suchbegriffe (Jobtitel) jeweils ein anderen Speicherort
try:
    suchbegriffe = pd.read_excel(r"C:\Users\Admin\DreamJobs\Benötigte Dateien\Jobtitel.xlsx",
                                sheet_name='LinkedIn')
except:
    suchbegriffe = pd.read_excel(r"C:\Users\Admin\DreamJobs\Benötigte Dateien\Jobtitel.xlsx",
                                sheet_name='LinkedIn')  
display(suchbegriffe)


KeyboardInterrupt



In [3]:
print(f'{len(suchbegriffe)} Jobtitel gefunden')
for i, jobtitel in enumerate(suchbegriffe.Jobtitel):
    print(f'{i}: Suche nach "{jobtitel}", Start: {datetime.datetime.now().strftime("%H:%M:%S")} Uhr')
    try:
        scraper(jobtitel)
    except UnboundLocalError as e:
        schreibe_e_mail(f'Scraper abgebrochen\nJobtitel: {jobtitel}\n    Zeit: {datetime.datetime.now()}\n\nFehler:\n{e}')
        schreibe_log_file(f'Scraper abgebrochen; Jobtitel: {jobtitel}')
        print(f'Scraper abgebrochen; Jobtitel: {jobtitel}')
print("")
print("fertig")

7 Jobtitel gefunden
0: Suche nach "Data Analyst", Start: 14:20:15 Uhr
1: Suche nach "Datenanalyst:in", Start: 14:50:58 Uhr
2: Suche nach "Business Intelligence Analyst ", Start: 14:51:29 Uhr
3: Suche nach "Junior Data Analyst", Start: 15:23:24 Uhr
4: Suche nach "Data-Specialist", Start: 15:23:53 Uhr
5: Suche nach "Data Architekt", Start: 15:24:20 Uhr
6: Suche nach "Trainee Data & Analytics", Start: 15:24:46 Uhr


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=116.0.5845.111)
Stacktrace:
	GetHandleVerifier [0x011D37C3+48947]
	(No symbol) [0x01168551]
	(No symbol) [0x0106C92D]
	(No symbol) [0x010536E5]
	(No symbol) [0x010B8CDB]
	(No symbol) [0x010C7113]
	(No symbol) [0x010B4E86]
	(No symbol) [0x010916C7]
	(No symbol) [0x0109284D]
	GetHandleVerifier [0x0141FDF9+2458985]
	GetHandleVerifier [0x0146744F+2751423]
	GetHandleVerifier [0x01461361+2726609]
	GetHandleVerifier [0x01250680+560624]
	(No symbol) [0x0117238C]
	(No symbol) [0x0116E268]
	(No symbol) [0x0116E392]
	(No symbol) [0x011610B7]
	BaseThreadInitThunk [0x76CB00C9+25]
	RtlGetAppContainerNamedObjectPath [0x77D37B1E+286]
	RtlGetAppContainerNamedObjectPath [0x77D37AEE+238]
